In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

from tqdm import tqdm
import pickle

In [2]:
from RegimeSwitch import hmm_states, calc_rs_params
from MeanShrinkage import bayes_stein_shrinkage
from CovShrinkage import LW_common_diag, LW_common_corr, LW_sample_diag, LW_common_vcov, denard_gcvc
from MVO import calc_mvo
from PerfStats import summary_stats

In [3]:
with open("dat/bbg/d_bbg_ret.pickle", "rb") as f:
    d_bbg_ret = pickle.load(f)
ret_jpy = d_bbg_ret["ret_jpy"]
ret_ir = d_bbg_ret["ret_ir"]

In [4]:
ret_jpy_deg = ret_jpy.copy()
ret_jpy_deg.columns = ret_jpy.columns.get_level_values(1)

# ret_jpy_ex_cash = ret_jpy.loc[:, [c!="Cash" for c in ret_jpy.columns.get_level_values(3)]]
# ret_jpy_ex_cash_deg = ret_jpy_ex_cash.copy()
# ret_jpy_ex_cash_deg.columns = ret_jpy_ex_cash.columns.get_level_values(1)

In [5]:
RISK_TOLERANCE = .25
WINDOW = 36

In [6]:
idx_dat = pd.read_csv("dat/bbg/acwi_wgbi.csv", index_col=0, skiprows=5, parse_dates=True)\
                .groupby(pd.Grouper(freq="BM")).last()
idx_ret = idx_dat.pct_change().dropna()

### Physical Asset and Currency Hedge

In [7]:
PARAM_COMBS = {
    ## Low Risk appetite
    # Sample Mean
    "Sample-Sample": ["sample", "sample"], 
    "Sample-LWCD": ["sample", "lwcd"], 
    "Sample-LWCC": ["sample", "lwcc"], 
    "Sample-GCVC": ["sample", "gcvc"], 
    "Sample-RS": ["sample", "regime_switch"], 
    # Bayes-Stein
    "BS-Sample": ["bayes-stein", "sample"], 
    "BS-LWCD": ["bayes-stein", "lwcd"], 
    "BS-LWCC": ["bayes-stein", "lwcc"], 
    "BS-GCVC": ["bayes-stein", "gcvc"], 
    "BS-RS": ["bayes-stein", "regime_switch"], 
    # Regime Switch
    "RS-Sample": ["regime_switch", "sample"], 
    "RS-LWCD": ["regime_switch", "lwcd"], 
    "RS-LWCC": ["regime_switch", "lwcc"], 
    "RS-GCVC": ["regime_switch", "gcvc"], 
    "RS-RS": ["regime_switch", "regime_switch"], 
}

In [8]:
eret_dict = {}; cov_dict = {}; shrinkage_intensity_dict = {}
wgt_dict = {}; attr_dict = {}; pfret_dict = {}

# asset_ub = [int(fx_tf!="Cash") for fx_tf in ret_jpy.columns.get_level_values(3)] # ub=1 for physical asset, 0 for fx
# asset_lb = [int(fx_tf!="Cash")-1 for fx_tf in ret_jpy.columns.get_level_values(3)] # lb=0 for physical asset, -1 for fx
# currency_group = pd.get_dummies(ret_jpy.columns.get_level_values(2)).values.T
# sum_one_asset = asset_ub # use asset_ub because 1 corresponds to physical assets and 0 to fx
# sum_one_asset = [int(fx_tf!="Cash") for fx_tf in ret_jpy.columns.get_level_values(3)]
# w_start = ret_jpy.index[WINDOW-1]


asset_ub = [1]*ret_jpy.shape[1]
asset_lb = [0]*ret_jpy.shape[1]
sum_one_asset = asset_ub
w_start = ret_jpy.index[WINDOW-1]


for dt in tqdm(ret_jpy.loc[w_start:,:].index):
    ret_jpy_win = ret_jpy.loc[:dt,:].tail(WINDOW)
    # params
    eret_bs_win = bayes_stein_shrinkage(ret_jpy_win)
    lwcd_win = LW_common_diag(ret_jpy_win)
    lwcc_win = LW_common_corr(ret_jpy_win)
    gcvc_win = denard_gcvc(ret_jpy_win, multi_class=True, group_level=3) # level_three for assetclass grp
    regime_switch_win = calc_rs_params(asset_return=ret_jpy_win, factor_return=idx_ret, hmm_window="expanding")

    # mean
    eret_dict[dt] = {
        "sample": ret_jpy_win.mean(),
        "bayes-stein": eret_bs_win[1],
        "regime_switch": regime_switch_win[0]
    }

    # cov
    cov_dict[dt] = {
        "sample": ret_jpy_win.cov(),
        "lwcd": lwcd_win[1],
        "lwcc": lwcc_win[1],
        "gcvc": gcvc_win[1],
        "regime_switch": regime_switch_win[1]
    }

    # method prop
    shrinkage_intensity_dict[dt] = {
        "bayes-stein": eret_bs_win[0],
        "lwcd": lwcd_win[0],
        "lwcc": lwcc_win[0],
        "gcvc": gcvc_win[0]
    }


    for k,v in PARAM_COMBS.items():
        wgt_dict[dt, k] = calc_mvo(eret_dict[dt][v[0]], cov_dict[dt][v[1]], risk_tolerance=RISK_TOLERANCE,
                                   asset_weight_upper_bound=asset_ub, asset_weight_lower_bound=asset_lb, 
                                   sum_one_asset=sum_one_asset)

wgt_mvo = pd.concat(wgt_dict, axis=0, names=["Date", "Strat"]).unstack(["Strat", "Ticker", "Currency", "AssetClass", "Name"])
attr = wgt_mvo.shift().mul(ret_jpy_deg).dropna()
pfret = attr.groupby(axis=1, level=0).sum().dropna()

100%|██████████| 249/249 [02:27<00:00,  1.69it/s]


In [9]:
ShrinkageRS_mvo_dict = {
    "eret": eret_dict,
    "cov":cov_dict,
    "shrinkage": shrinkage_intensity_dict,
    "wgt": wgt_mvo,
    "attr": attr,
    "ret": pfret
}

with open("output/ShrinkageRS/ShrinkageRS_mvo_dict_jpy.pickle", "wb") as f:
    pickle.dump(ShrinkageRS_mvo_dict, f)